In [ ]:

from scipy.io import readsav
import keras.backend as b
import pandas as pd
import keras as K
import tensorflow as tf
import datetime
import numpy as np
import math
import os
import joblib
# from sklearn import cross_validation
import matplotlib.pyplot as plt
from sklearn import preprocessing
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.callbacks import LearningRateScheduler
from keras.layers.core import Dense, Dropout
from sklearn import preprocessing
# from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8 
config.gpu_options.allow_growth = True    
sess = tf.compat.v1.Session(config = config)

In [ ]:
import numpy as np

file_path1 = '/shuju/全部数据的训练集.csv'
file_path2 = '/shuju/全部数据的测试集.csv'

data = np.genfromtxt(file_path1, delimiter=',', skip_header=1) 
data_test = np.genfromtxt(file_path2, delimiter=',', skip_header=1)

np.save('/shuju/fangan3data2mev.npy', data)
np.save('/shuju/2015-10-12.npy', data_test)

In [ ]:
import numpy as np
import pandas as pd

input_file_path1 = '/shuju/fangan3data2mev.npy'  
input_file_path2 = '/shuju/2015-10-12.npy'
data1 = np.load(input_file_path1)
datax = data1[:,1:50]                     
datay = np.reshape(data1[:, 50],(-1,1))   
print(datax.shape)
print(datay.shape)
data3 = np.load(input_file_path2)
datax_test = data3[:,1:50]                    
datay_test = np.reshape(data3[:, 50],(-1,1))    

In [ ]:
datax = mm.fit_transform(datax)
datay = mm.fit_transform(datay)
datax_test = mm.fit_transform(datax_test)
datay_test = mm.fit_transform(datay_test)

In [ ]:
x_data = np.array(datax)
y_data = np.array(datay)

x_test = np.array(datax_test)
y_test = np.array(datay_test)

In [ ]:
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

# 定义预测效率（PE）评估指标
def pe(y_true, y_pred):
    return 1 - (K.sum(K.square(y_pred - y_true)) / K.sum(K.square(y_true - K.mean(y_true))))

# 定义相关系数（CC）评估指标
def cc(y_true, y_pred):
    mean_true = K.mean(y_true)
    mean_pred = K.mean(y_pred)
    covar = K.mean((y_true - mean_true) * (y_pred - mean_pred))
    std_true = K.std(y_true)
    std_pred = K.std(y_pred)
    return covar / (std_true * std_pred)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.initializers import he_normal

def pred_model():
    model = Sequential()  
    model.add(Dense(49, kernel_initializer=he_normal(), activation='relu', input_shape=(49,)))
    model.add(Dense(25, kernel_initializer=he_normal(), activation='relu'))
    model.add(Dropout(0.2)) 
    model.add(Dense(1))
    return model
model = pred_model()
print(model.summary())


In [ ]:
model = pred_model()  
reduce_lr = ReduceLROnPlateau(monitor='pe',    
                             factor=0.7,         
                             patience=20,       
                             mode='auto',      
                             verbose=1)          

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping

model = pred_model()  
early_stopping = EarlyStopping(monitor='val_loss', 
                               patience=20, 
                               verbose=1, 
                               restore_best_weights=True)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint

group_size = 288
num_groups = len(x_data) // group_size

num_folds = 12
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

all_train_losses = []
all_val_losses = []
k_fold_pe_values = []
weights_list = []

for fold, (train_groups, val_groups) in enumerate(kf.split(range(num_groups))):   
    print(f"在第 {fold + 1}/{num_folds} 折上进行训练")
    train_indices = [idx for group_idx in train_groups for idx in range(group_idx * group_size, (group_idx + 1) * group_size)]
    val_indices = [idx for group_idx in val_groups for idx in range(group_idx * group_size, (group_idx + 1) * group_size)]
    x_train, y_train = x_data[train_indices], y_data[train_indices]
    x_val, y_val = x_data[val_indices], y_data[val_indices]
    model.compile(optimizer='adam', loss='mse', metrics=[rmse, pe, cc])
    checkpoint_filepath = f"/shuju/model/fangan32mev_model_fold_{fold}.h5"

    model_checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                        save_best_only=True, 
                                        save_weights_only=True, 
                                        monitor='val_loss', 
                                        mode='min',          
                                        verbose=1)    
    history = model.fit(x_train, 
                        y_train, 
                        epochs=100, 
                        batch_size=500, 
                        validation_data=(x_val, y_val),
                        verbose=1, 
                        shuffle=False, 
                        callbacks=[reduce_lr, model_checkpoint])

    all_train_losses.append(history.history['loss'])
    all_val_losses.append(history.history['val_loss'])
    k_fold_pe_values.append(history.history['val_pe'])

    weights = model.get_weights()
    weights_list.append(weights)

average_weights = np.mean(weights_list, axis=0)
model.set_weights(average_weights)
average_weights_filepath = "/shuju/fangan32mev_average_weights.h5"
model.save_weights(average_weights_filepath)
print(f"平均权重已保存至 {average_weights_filepath}")


In [ ]:

import matplotlib.pyplot as plt
import numpy as np

miniepoch = 20
all_train_losses = [epoch_data[:miniepoch] for epoch_data in all_train_losses]
all_val_losses = [epoch_data[:miniepoch] for epoch_data in all_val_losses]

all_train_losses = np.array(all_train_losses)
all_val_losses = np.array(all_val_losses)

mean_train_loss = np.mean(np.array(all_train_losses), axis=0)
mean_val_loss = np.mean(np.array(all_val_losses), axis=0)
std_train_loss = np.std(np.array(all_train_losses), axis=0)
std_val_loss = np.std(np.array(all_val_losses), axis=0)


c = range(miniepoch)
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(c, mean_train_loss, "b-", linestyle='solid', label="mean_train_loss")
ax.plot(c, mean_val_loss, "r-", linestyle='solid', label="mean_val_loss")

ax.fill_between(c, mean_train_loss - std_train_loss, mean_train_loss + std_train_loss, color='blue', alpha=0.3)
ax.fill_between(c, mean_val_loss - std_val_loss, mean_val_loss + std_val_loss, color='red', alpha=0.3)

ax.legend()
ax.set_xlabel('epoch')
ax.set_ylabel('value')
ax.set_title('2mev: Compare mean_train_loss and mean_val_loss with standard deviation')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

k_fold_pe_values = [epoch_data[:miniepoch] for epoch_data in k_fold_pe_values]
k_fold_pe_values = np.array(k_fold_pe_values)

# 计算平均值和标准差
mean_pe_values = np.mean(k_fold_pe_values, axis=0)
std_pe_values = np.std(k_fold_pe_values, axis=0)

epochs = range(1, len(mean_pe_values) + 1)

plt.plot(epochs, mean_pe_values, marker='o', linestyle='-', color='b', label='mean_PE')
plt.fill_between(epochs, mean_pe_values - std_pe_values, mean_pe_values + std_pe_values, color='blue', alpha=0.3, label='std_dev_PE')

plt.title('2mev: PE vs. Epoch')
plt.xlabel('Epoch')
plt.ylabel('PE')
plt.legend()
plt.grid(True)
plt.show()
